In [1]:
from tensorflow import keras
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications import mobilenet

In [2]:
def resnet50_model():
    model_input_shape = (224,224)
    model = resnet50.ResNet50(include_top=True, weights='imagenet', input_shape=(224, 224, 3), classes=1000)
    model = keras.Model(inputs=model.input, outputs=model.get_layer(index=-2).output)
    return (model, model_input_shape)

In [3]:
def inceptionv3_model():
    model_input_shape = (299, 299) 
    model = inception_v3.InceptionV3(include_top=True, weights='imagenet', input_shape=(299, 299, 3), classes=1000)
    model = keras.Model(inputs=model.input, outputs=model.get_layer(index=-2).output)
    return (model, model_input_shape)

In [1]:
def mobilenet_model():
    model_input_shape = (224, 224) 
    model = mobilenet.MobileNet(include_top=True, weights='imagenet', input_shape=(224, 224, 3), classes=1000)
    model = keras.Model(inputs=model.input, outputs=model.get_layer(index=-2).output)
    return (model, model_input_shape)